In [1]:
##import libraries
from tinyimagenet import TinyImageNet
from pathlib import Path
import torch
import torch.optim as optim
from torchvision import models
import torch.utils.data as data
from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
from torchvision import transforms as T

import poutyne

import pandas as pd
import os

import numpy as np
from pylab import *
import sys
import random

#/Users/tatibada/Documents/Tesis Maestria DM/scripts/EfficientNet/pipeline_B0_weightsnone_soloentrenamiento_v2.py
#sys.path.append('/content/drive/MyDrive/Tesis de Ms Data Mining TBadaracco/Notebooks/')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# load transformations
import transformaciones as tr

rotation_transforms = tr.rotation_transforms()
translation_transforms = tr.translation_transforms()
scale_transforms = tr.scale_transforms()
perspective_transforms = tr.perspective_transforms()
brightness_transforms = [tr.brightness_transforms(factor) for factor in tr.brightness_parameters]
contrast_transformations = [tr.contrast_transforms(alpha) for alpha in tr.contrast_list]
grayscale_transformations = [tr.grayscale_transforms(alpha) for alpha in tr.grey_list]
solarize_transformations = [tr.solarize_transforms(threshold) for threshold in tr.solarization_thresholds]
posterize_transformations = [tr.posterize_transforms(alpha) for alpha in tr.posterize_list]
invertion_transformations = [tr.invertion_transforms(alpha) for alpha in tr.invertion_list]


transformation_afin = [rotation_transforms,
                       translation_transforms,
                       scale_transforms,
                       perspective_transforms,
                       brightness_transforms,
                       contrast_transformations,
                       grayscale_transformations,
                       solarize_transformations,
                       posterize_transformations,
                       invertion_transformations]


transformaciones = ['rotacion','traslacion','escala','proyeccion','brillo','contraste','escala_grises','solarizacion','posterizacion','inversion_colores']

print('import transformations')

#### para salucionar error: RuntimeError: invalid hash value (expected "7eb33cd5", got "23ab8bcd5bdbef61a7a43b91adcad81f622fd7f36fb4935a569828d77888c44e")

import transformations


In [3]:
## si quiero con vit b16 de pytorch
# def get_state_dict(self, *args, **kwargs):
#     kwargs.pop("check_hash")
#     return load_state_dict_from_url(self.url, *args, **kwargs)

# WeightsEnum.get_state_dict = get_state_dict
# #####

# # Definir el modelo y checkpoint
# weights = models.ViT_B_16_Weights.IMAGENET1K_V1
# base_model = models.vit_b_16(weights=weights)

In [7]:

from timm.models import _builder

_builder._update_default_kwargs = _builder._update_default_model_kwargs

## fastervit
from fastervit import create_model
# Define fastervit-0 model with 224 x 224 resolution

base_model = create_model('faster_vit_0_224', 
                          pretrained=True,
                          model_path="/home/tbadaracco//faster_vit_0.pth.tar")

/home/tbadaracco/tb_env/lib/python3.10/site-packages/fastervit/models/faster_vit.py:193: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(filename, map_

In [4]:
base_model

FasterViT(
  (patch_embed): PatchEmbed(
    (proj): Identity()
    (conv_down): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (levels): ModuleList(
    (0): FasterViTLayer(
      (blocks): ModuleList(
        (0): ConvBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act1): GELU(approximate='none')
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [8]:


for param in base_model.parameters():
    param.requires_grad = True

# Descongelar los parámetros de la capa de clasificación
# for param in base_model.heads.parameters():
#     param.requires_grad = True

tinyimagenet_classes = 200
# base_model.classifier = torch.nn.Sequential(
#     torch.nn.Dropout(p=0.2, inplace=True),
#     torch.nn.Linear(1280, tinyimagenet_classes),
# )
# Reemplazar el clasificador del modelo FasterViT
base_model.head = torch.nn.Sequential(
    torch.nn.Dropout(p=0.2, inplace=True),
    torch.nn.Linear(512, tinyimagenet_classes),  # Ajustar el tamaño según la salida de la capa anterior
)

# model = torch.nn.Sequential(
#    #T.Normalize(TinyImageNet.mean,TinyImageNet.std),
#     weights.transforms(),
#     base_model,
# )


model = base_model.to(device)



In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
#scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min')

# Directorio principal donde se guardarán los resultados
main_dir = 'models/ViT/fine_tuning_v2'

# Crear directorio principal si no existe
Path(main_dir).mkdir(parents=True, exist_ok=True)

num_epochs = 20

class TinyImageNet(TinyImageNet):
    def __getitem__(self, index):
        x, y = super().__getitem__(index)
        return x,y
#transformaciones = ['Without_transformation']
# Iterar sobre cada transformación
for i, transformacion in enumerate(transformaciones):
    print(f"Training with transformation: {transformacion}")

    # Crear directorio para la transformación actual
    transform_dir = os.path.join(main_dir, transformacion)
    Path(transform_dir).mkdir(parents=True, exist_ok=True)

    # Definir el path del checkpoint
    checkpoint_path = os.path.join(transform_dir, 'checkpoint_last.ckpt')
    checkpoint_opt_path = os.path.join(transform_dir, 'checkpoint_opt.ckpt') 

    print(f'checkpoint path: {checkpoint_path}')
    print(f'checkpoint optimizer path: {checkpoint_opt_path}')

    # Cargar o reiniciar el modelo y el checkpoint para cada transformación
    if os.path.exists(checkpoint_path):
        model.load_state_dict(torch.load(checkpoint_path, map_location='cuda' if torch.cuda.is_available() else 'cpu'))

    # Carga el estado del optimizador desde el archivo
    if os.path.exists(checkpoint_opt_path):
        optimizer.load_state_dict(torch.load(checkpoint_opt_path, map_location='cuda' if torch.cuda.is_available() else 'cpu'))

    # Definir el conjunto de datos con la transformación actual
    current_transform = transformation_afin[i]
    #print(current_transform)

    random_ts = lambda x: random.choice(current_transform)(x)
    
    normalize_transform = T.Compose(
        [
        T.Resize((224, 224)),
        T.ToTensor(),
        T.transforms.Normalize(TinyImageNet.mean,TinyImageNet.std),
        #random_ts
        ])

    # Definir el conjunto de datos original
    dataset_train = TinyImageNet(Path("~/.torchvision/tinyimagenet/"), split="train", imagenet_idx=False, transform=normalize_transform)
    dataset_val = TinyImageNet(Path("~/.torchvision/tinyimagenet/"), split="val", imagenet_idx=False, transform=normalize_transform)

    #print(dataset_train[0])
    # Definir un DataLoader para cargar los datos originales por lotes
    train_loader = data.DataLoader(dataset_train, batch_size=32, shuffle=True)
    val_loader = data.DataLoader(dataset_val, batch_size=32, shuffle=True)


    print('DataLoader listo')


    # Definir el trainer
    trainer = poutyne.Model(
        model,
        optimizer,
        'cross_entropy',
        batch_metrics=['accuracy', poutyne.TopKAccuracy(5)],
        epoch_metrics=['f1'],
        device=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    )

    # Definir el historial y el checkpoint para cada transformación
    history_path = os.path.join(transform_dir, 'history.csv')
    checkpoint_path = os.path.join(transform_dir, 'checkpoint_last.ckpt')
    best_checkpoint_path = os.path.join(transform_dir, 'tmp_best.ckpt')
    checkpoint_opt_path = os.path.join(transform_dir, 'checkpoint_opt.ckpt')   

    checkpoint = poutyne.ModelCheckpoint(checkpoint_path, monitor='val_acc', mode='max', save_best_only=False, restore_best=False, verbose=True, temporary_filename=best_checkpoint_path)
    opt_checkpoint = poutyne.OptimizerCheckpoint(checkpoint_opt_path, monitor='val_acc', mode='max', save_best_only=False, restore_best=False, verbose=True,        temporary_filename=best_checkpoint_path)

    class HistorySaver(poutyne.Callback):

      def __init__(self,filepath):
        super().__init__()
        self.filepath = filepath
        self.history = []

      def on_epoch_end(self, epoch, logs):
        self.history.append(logs)

        if os.path.exists(history_path):
          df1 = pd.read_csv(history_path)
          df = pd.DataFrame(self.history)
          df = pd.concat([df1,df])
          df.reset_index(drop=True,inplace=True)
          df.drop_duplicates(inplace = True, ignore_index = True)
          df.to_csv(self.filepath,index=False)
        else:
          df = pd.DataFrame(self.history)
          df.to_csv(self.filepath,index=False)

    # callback personalizado
    history_saver = HistorySaver(history_path)

    if os.path.exists(history_path):
        df1 = pd.read_csv(history_path)
        history_saver.history = df1.to_dict('records')

    # Entrenar el modelo para cada transformación
    #history = trainer.fit_dataset(train_loader, valid_dataset=val_loader, batch_size=8, epochs=num_epochs, callbacks=[checkpoint, opt_checkpoint,history_saver])

    history = trainer.fit_generator(train_loader, val_loader, epochs=num_epochs, callbacks=[checkpoint, opt_checkpoint, history_saver])

Training with transformation: Without_transformation
checkpoint path: models/ViT/fine_tuning_v2/Without_transformation/checkpoint_last.ckpt
checkpoint optimizer path: models/ViT/fine_tuning_v2/Without_transformation/checkpoint_opt.ckpt
DataLoader listo
Epoch:  1/20 Train steps: 3125 Val steps: 313 14m45.74s loss: 2.765247 acc: 35.363000 top5: 62.534000 fscore_macro: 0.346507 val_loss: 1.895455 val_acc: 52.720000 val_top5: 79.150000 val_fscore_macro: 0.520790
Epoch 1: saving file to models/ViT/fine_tuning_v2/Without_transformation/checkpoint_last.ckpt
Epoch 1: saving file to models/ViT/fine_tuning_v2/Without_transformation/checkpoint_opt.ckpt
Epoch:  2/20 Train steps: 3125 Val steps: 313 14m46.59s loss: 1.826214 acc: 54.216000 top5: 80.401000 fscore_macro: 0.539103 val_loss: 1.734774 val_acc: 56.410000 val_top5: 82.020000 val_fscore_macro: 0.563078
Epoch 2: saving file to models/ViT/fine_tuning_v2/Without_transformation/checkpoint_last.ckpt
Epoch 2: saving file to models/ViT/fine_tuning